In [44]:
from twelvedata import TDClient
from twelvedata.exceptions import BadRequestError


ticker = 'TSLA'
interval = '1min'
output_size = 5000
year = 2020
filename = f'{ticker}_{year}_{interval}.csv'

# This is an API limit for TwelveData, 8 API calls per minute.
# For better rate limiting, i'm lowering the limit on our end
# just so we're not even close to it.
# Each iteration is technically 3 API calls.
iterations_per_minute=2


In [47]:
import sys

# Load Twelve Data API key
api_key = None

with open('twelve_data_api_key', 'r') as file:
    api_key = file.read()
    
if api_key is None:
    sys.exit("Error, no API key")

In [47]:
import calendar
import time

# Let's fill this in with a month-by-month dateranges to feed to the API
# This is broken up just so we don't fill up the 5000 outputsize maximum APIlimit.
date_ranges = []

# 12 entries starting from 1 for the month
# calendar.monthrange gives us (start-date, end-date) for the month in question
for month in range(1, 13):
    daterange = calendar.monthrange(year, month)
    date_ranges.append((f'{year}-{month}-01', f'{year}-{month}-{daterange[1]}'))

# Because the output is ordered descending in date, let's reverse this list so we get december first.
date_ranges.reverse()

td = TDClient(apikey=api_key)
csv_data = []
csv_headers = None
df_list = []
iteration_count = 0

for date_range in date_ranges:
    print(f'Fetching data for date range: {date_range}.. ', end='')

    try:
        ts = td.time_series(symbol=ticker,
                            interval=interval,
                            start_date=date_range[0],
                            end_date=date_range[1],
                            outputsize=output_size)

        ts_csv = ts.as_csv()
        if csv_headers == None:
            csv_headers = ts_csv[0]

        csv_data.extend(ts_csv[1:])
        df_list.append(ts.as_pandas())

    except BadRequestError as e:
        print(e)

    finally:
        print('done')
        iteration_count += 1
        if (iteration_count % iterations_per_minute) == 0:
            time.sleep(60)

csv_data.insert(0, csv_headers)

Fetching data for date range: ('2020-12-01', '2020-12-31').. done
Fetching data for date range: ('2020-11-01', '2020-11-30').. done
Fetching data for date range: ('2020-10-01', '2020-10-31').. done
Fetching data for date range: ('2020-9-01', '2020-9-30').. done
Fetching data for date range: ('2020-8-01', '2020-8-31').. done
Fetching data for date range: ('2020-7-01', '2020-7-31').. done
Fetching data for date range: ('2020-6-01', '2020-6-30').. done
Fetching data for date range: ('2020-5-01', '2020-5-31').. done
Fetching data for date range: ('2020-4-01', '2020-4-30').. done
Fetching data for date range: ('2020-3-01', '2020-3-31').. done
Fetching data for date range: ('2020-2-01', '2020-2-29').. No data is available on the specified dates. Try setting different start/end dates.
done
Fetching data for date range: ('2020-1-01', '2020-1-31').. No data is available on the specified dates. Try setting different start/end dates.
done


In [48]:
# Write data to CSV file.
import csv
with open(filename, 'w') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)
file.close()